In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os
import seaborn
from tqdm import tqdm

trainPath = 'data/Dataset_1_train_asset.csv'
targetPath = 'data/Dataset_1_train_payoff.csv'

trainData = pd.read_csv(trainPath)
trainData.columns = [i for i in range(0, 31)]
target = pd.read_csv(targetPath)

In [ ]:
# exercise 1 - BlackScholes Model

N = norm.cdf

def bs_call(K: float, #underlying asset's price
            T: float, #time till maturity
            S: float = 1.0, # strike price
            r: float = 0.0,  # risk-free
            sigma: float = 0.158):
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2), 1 - N(d1)

def bs_put(K: float, #underlying asset's price
           T: float, #time till maturity
           S: float = 1.0, #strike price
           r: float = 0.0, # risk-free
           sigma: float = 0.158):
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

In [ ]:
def calculate_hedge(df: pd.DataFrame):
    deltas = pd.DataFrame(index=range(df.shape[0]),columns=range(df.shape[1]))
    portfolio_value_50 = pd.DataFrame(index=range(df.shape[0]),columns=range(df.shape[1]))
    portfolio_value_99 = pd.DataFrame(index=range(df.shape[0]),columns=range(df.shape[1]))
    
    for i in tqdm(range(0, df.shape[0])):
        short_position = 0.0
        for j in range(0, df.shape[1]):
            S = 100
            initial_price = 100
            K = df.loc[i, :].values[j]
            T = (31 - j)/365
            result, delta = bs_call(K=K, T=T, S=S)
            short_position += -result
            deltas.loc[i, j] = delta
            option_value = max(0, S - K)
            pnl_50 = (option_value + short_position*S) - initial_price*0.5
            pnl_99 = (option_value + short_position*S) - initial_price*0.01
            portfolio_value_50.loc[i, j] = pnl_50
            portfolio_value_99.loc[i, j] = pnl_99
    return deltas, portfolio_value_50, portfolio_value_99


In [ ]:
bs_deltas, val50, val99 = calculate_hedge(trainData)

In [ ]:
def plot_figures(deltas: pd.DataFrame, df: pd.DataFrame): 
    fig, axs = plt.subplots(2, 3)
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle('Delta spread at different time intervals', size=45)

    axs[0,0].plot(df.loc[:, 0], deltas.loc[:, 0], 'bo')
    axs[0,0].set_title('Time 0')
    axs[0,1].plot(df.loc[:, 1], deltas.loc[:, 1], 'bo')
    axs[0,1].set_title('Time 1')
    axs[0,2].plot(df.loc[:, 5], deltas.loc[:, 5], 'bo')
    axs[0,2].set_title('Time 5')
    axs[1,0].plot(df.loc[:, 15], deltas.loc[:, 15], 'bo')
    axs[1,0].set_title('Time 15')
    axs[1,1].plot(df.loc[:, 25], deltas.loc[:, 25], 'bo')
    axs[1,1].set_title('Time 25')
    axs[1,2].plot(df.loc[:, 30], deltas.loc[:, 30], 'bo')
    axs[1,2].set_title('Time 30')

    for ax in axs.flat:
        ax.set(xlabel='Spot price', ylabel='Delta')

In [ ]:
plot_figures(bs_deltas, trainData)